# PAN16

In [1]:
import os
from argparse import ArgumentParser
from sklearn.externals import joblib
from tictacs import from_recipe
from pan import ProfilingDataset
import dill
import cPickle as pickle
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix


infolder = "../DATA/pan16-author-profiling-training-dataset-2016-04-25/pan16-author-profiling-training-dataset-english-2016-02-29/"
outfolder = "models/"
print('Loading dataset->Grouping User texts.\n')
dataset = ProfilingDataset(infolder)
print('Loaded {} users...\n'.format(len(dataset.entries)))
# get config
config = dataset.config
tasks = config.tasks
print('\n--------------- Thy time of Running ---------------')
for task in tasks:
    print('Learning to judge %s..' % task)
    # load data
    X, y = dataset.get_data(task)

Loading dataset->Grouping User texts.

Loaded 436 users...


--------------- Thy time of Running ---------------
Learning to judge age..
Learning to judge gender..


### Instance Based

In [ ]:
from pan import ProfilingDataset, createDocProfiles, create_target_prof_trainset
from pan import preprocess

task = 'gender'
docs = createDocProfiles(dataset)
X, y = create_target_prof_trainset(docs, task)
print len(X)
X = preprocess.preprocess(X)

### Profile Based

In [2]:
#reload(preprocess)
#reload(features)
from pan import features
from pan import preprocess
X, y = dataset.get_data('age')
#X, y = dataset.get_data('gender')
print len(X)
#print X[0]
X = preprocess.preprocess(X)
#print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
#print X[0]

436


# Models

### AGE

In [3]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from pan.features import SOA_Model2

grams3 = TfidfVectorizer(analyzer='word', ngram_range=[2,2], max_features=5000, stop_words='english')
soa = features.SOA_Model2(max_df=1.0, min_df=1, tokenizer_var='sklearn', max_features=None)
combined = FeatureUnion([('3grams', grams3), ('soa', soa)])
svm = SVC(kernel='rbf', C=10, gamma=1, class_weight='balanced', probability=False)
pipe = Pipeline([('3grams',grams3), ('svm', svm)])
pipe.steps

[('3grams',
  TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
          dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
          lowercase=True, max_df=1.0, max_features=5000, min_df=1,
          ngram_range=[2, 2], norm=u'l2', preprocessor=None, smooth_idf=True,
          stop_words='english', strip_accents=None, sublinear_tf=False,
          token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('svm', SVC(C=10, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

### SOAC

In [4]:
soac = features.SOAC_Model2(max_df=1.0, min_df=1, tokenizer_var='sklearn', max_features=None)
svm = SVC(kernel='rbf', C=1, gamma=1, class_weight='balanced', probability=False)
#combined = FeatureUnion([('count_tokens', countTokens), ('count_hash', countHash),
#                         ('count_urls', countUrls), ('count_replies', countReplies), 
#                          ('soa', soa), ('soac', soac)])+
#combined = FeatureUnion([('count_tokens', countTokens), ('count_hash', countHash),
#                         ('count_urls', countUrls), ('count_replies', countReplies)])
pipe1 = Pipeline([('soac',soac), ('svm', svm)])
pipe1.steps

[('soac', SOAC_Model2(max_df=1.0, max_features=None, min_df=1, thres=0.1,
        tokenizer_var='sklearn')),
 ('svm', SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

### LSI

In [5]:
from pan.features import LSI_Model
LSImodel = LSI_Model(num_topics=100)
svm = SVC(kernel='rbf', C=0.1, gamma=1, class_weight='balanced', probability=False)
#pipe2 = Pipeline([('counts',combined), ('svm', svm)])
pipe2 = Pipeline([('LSI',LSImodel), ('svm', svm)])

# Train

In [6]:
def print_overlaps(predictions, names, verbose=True):
    N = len(names)
    res = numpy.zeros([N,N])
    temp = numpy.zeros([N,N])
    for i in range(0, N):
        for j in range(i+1, N):
            temp[i,j] = len([m for l, m in enumerate(predictions[i]) if (m==predictions[j][l] and m==predictions[N-1][l])])/float(len(predictions[0]))
            res[i,j] = len([(k,v) for k,v in zip(predictions[i], predictions[j]) if k==v])/float(len(predictions[0]))
            if verbose:
                print "%s - %s : %0.3f  overlap | ground-truth coverage: %0.3f" % (names[i],  names[j], 100*res[i,j], 100*temp[i,j])
    return  [res, temp]

In [ ]:
reload(combinations)
combinations.

In [21]:
import sys
sys.path.insert(0, "../EnsembleDiversityTests/")
import combinations
import copy
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,precision_recall_fscore_support, classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.grid_search import GridSearchCV
from pan.features import Metaclassifier
import time
import numpy

import warnings
warnings.filterwarnings('ignore')

#pipe = Pipeline([('3grams',grams3), ('svm', svm)])
#pipe1 = Pipeline([('soac',soac), ('svm', svm)])
#pipe2 = Pipeline([('soa',soa), ('svm', svm)])

### AGE ###
#eclf = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ("2", pipe2)], voting='soft')
#eclfh = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ("2", pipe2)], voting='hard')
#models = [pipe,pipe1,pipe2,eclf, eclfh]
#model_names = ['3grams', 'soac', 'lda', 'voting', 'votingh']

# Base Models
base_models = [pipe, pipe1, pipe2]
base_model_names = ['3grams', 'soac', 'lsi']

# Meta Voting Models
eclf = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ('2', pipe2)], voting='soft')
eclfh = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ('2', pipe2)], voting='hard')
voting_dic = {'votingf':eclf, 'votingh':eclfh}
combinator_names = ['majority', 'weights', 'accuracy', 'optimal']
#meta_models_names = ['votingf', 'votingh', 'space3', 'meta'] + combinator_names
meta_models_names = ['space3'] #+ combinator_names
## all_models ##
all_models_names = base_model_names + meta_models_names


#eclf = VotingClassifier(estimators=[("0", pipe), ('1', pipe1)], voting='soft')
#eclfh = VotingClassifier(estimators=[("0", pipe), ('1', pipe1)], voting='hard')
#models = [pipe,pipe1,eclf, eclfh]
#model_names = ['3grams', 'soac', 'voting', 'votingh']

results = {'over':[]}
for name in all_models_names:
    results[name] = {'pred': [], 'conf': [], 'rep': [], 'acc': []}

num_folds = 4
train_split = 0.3
meta_split = 0.5
cv_rounds = 10
t0 = time.time()
t1 = t0
for j in xrange(cv_rounds):
    X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=train_split, stratify=y)
    for i, x in enumerate(X_train):
        if len(x)==0:
            X_train.remove(x)
            y_train.remove(y_train[i])
    for i, x in enumerate(X_cv):
        if len(x)==0:
            X_cv.remove(x)
            y_cv.remove(y_cv[i])
    if meta_split > 0:
        X_meta, X_cv, y_meta, y_cv = train_test_split(X_cv, y_cv, test_size=meta_split, stratify=y_cv)
        print len(X_train), len(X_cv), len(X_meta), len(X_cv) + len(X_train) + len(X_meta), len(X)
    else:
        print len(X_train), len(X_cv), len(X_cv) + len(X_train) , len(X)
    trained_base_models = []
    predictions = []
    base_predictions_cv = []
    base_predictions_meta = []
    for i, model in enumerate(base_models):
        model.fit(X_train,y_train)
        trained_base_models.append(model)
        predict = model.predict(X_cv)
        predictions.append(predict)
        base_predictions_cv.append(predict)
        #base_predictions_meta.append(model.predict(X_meta))
        results[base_model_names[i]]['pred'].append(predict)
        results[base_model_names[i]]['acc'].append(accuracy_score(y_cv, predict))
        results[base_model_names[i]]['conf'].append(confusion_matrix(y_cv, predict, labels=list(set(y))))
        results[base_model_names[i]]['rep'].append(classification_report(y_cv, predict, labels=list(set(y))))
    trained_all_models = copy.deepcopy(trained_base_models)
    for name in meta_models_names:
        #print name
        if name =='votingf' or name=='votingh':
            model = voting_dic[name]
            model.fit(X_train, y_train)
            predict = model.predict(X_cv)
        if name == 'space':
            models_for_space = {}
            cv_scores = []
            for i, base_trained_model in enumerate(trained_base_models):
                models_for_space[base_model_names[i]] = base_trained_model
                cv_scores.append(base_trained_model.score(X_meta, y_meta))
            model = combinations.SubSpaceEnsemble4_2(models_for_space, cv_scores, k=6, weights=[0.65,0.35,0.32,6], N_rand=10, rand_split=0.6)
            model.fit(X_meta, y_meta)
            predict = model.predict(X_cv)
        if name == 'space3':
            models_for_space = {}
            for i, base_trained_model in enumerate(trained_base_models):
                models_for_space[base_model_names[i]] = base_trained_model
            model = SubSpaceEnsemble3(models_for_space, k=5, weights= [2,1,3,0.6])
            model.fit(X_train, y_train)
            predict = model.predict(X_cv)
        if name == 'meta':
            model_dic = {}
            for i, base_trained_model in enumerate(trained_base_models):
                model_dic[base_model_names[i]] = base_trained_model
            model = Metaclassifier(models=model_dic, C=1.0, weights='balanced')
            model.fit(X_meta, y_meta)
            predict = model.predict(X_cv)
        if name in combinator_names:
            #print 'mpike'
            model = combinations.Combinator(scheme=name, weights= [1/float(len(base_predictions_meta)) for i in xrange(len(base_predictions_meta))])
            model.fit(base_predictions_meta, y_meta)
            predict = model.predict(base_predictions_cv)
        trained_all_models.append(model)
        predictions.append(predict)
        results[name]['pred'].append(predict)
        results[name]['acc'].append(accuracy_score(y_cv, predict))
        results[name]['conf'].append(confusion_matrix(y_cv, predict, labels=list(set(y))))
        results[name]['rep'].append(classification_report(y_cv, predict, labels=list(set(y))))
    print('Round %d took: %0.3f seconds') % (j, time.time()-t1)
    t1 = time.time()
print('Total time: %0.3f seconds') % (time.time()-t0)

302 65 65 432 436
Fit took: 11.881 seconds
Predict took: 248.361 seconds
Round 0 took: 288.737 seconds
303 65 64 432 436
Fit took: 11.593 seconds
Predict took: 293.411 seconds
Round 1 took: 332.095 seconds
303 65 64 432 436
Fit took: 11.497 seconds
Predict took: 270.229 seconds
Round 2 took: 308.851 seconds
302 65 65 432 436
Fit took: 11.357 seconds
Predict took: 301.399 seconds
Round 3 took: 339.907 seconds
302 65 65 432 436
Fit took: 11.556 seconds
Predict took: 373.047 seconds
Round 4 took: 412.087 seconds
303 65 64 432 436
Fit took: 12.338 seconds
Predict took: 275.913 seconds
Round 5 took: 316.432 seconds
301 66 65 432 436
Fit took: 11.370 seconds
Predict took: 334.509 seconds
Round 6 took: 372.412 seconds
305 64 63 432 436
Fit took: 11.815 seconds
Predict took: 292.918 seconds
Round 7 took: 332.446 seconds
302 65 65 432 436
Fit took: 11.847 seconds
Predict took: 329.510 seconds
Round 8 took: 368.647 seconds
303 65 64 432 436
Fit took: 11.937 seconds
Predict took: 340.734 seconds


In [170]:
results['space3']['conf']

[array([[11,  6,  4,  0,  0],
        [ 5, 17,  2,  2,  1],
        [ 2,  5,  4,  1,  0],
        [ 0,  2,  0,  2,  0],
        [ 0,  0,  1,  0,  0]]), array([[19,  1,  1,  0,  0],
        [ 0, 27,  0,  0,  0],
        [ 0,  1, 11,  0,  0],
        [ 0,  0,  0,  4,  0],
        [ 0,  0,  0,  0,  1]]), array([[11,  8,  1,  0,  1],
        [ 3, 21,  2,  1,  0],
        [ 0,  4,  5,  2,  1],
        [ 1,  1,  0,  2,  0],
        [ 0,  0,  0,  1,  0]]), array([[10,  8,  2,  1,  0],
        [ 6, 16,  2,  3,  0],
        [ 2,  6,  4,  0,  0],
        [ 0,  1,  2,  1,  0],
        [ 1,  0,  0,  0,  0]]), array([[12,  4,  2,  2,  1],
        [ 1, 23,  2,  1,  0],
        [ 1,  6,  5,  0,  0],
        [ 0,  1,  2,  1,  0],
        [ 0,  0,  1,  0,  0]]), array([[18,  3,  0,  0,  0],
        [ 0, 28,  0,  0,  0],
        [ 0,  4,  8,  0,  0],
        [ 0,  0,  0,  4,  0],
        [ 0,  0,  0,  0,  1]]), array([[19,  2,  0,  0,  0],
        [ 1, 26,  0,  0,  0],
        [ 0,  2, 10,  0,  0],
    

In [22]:
for name in all_models_names:
    print '%%%%%%%%%%%%%%%%  ' + name  + '  % %%%%%%%%%%%%%%%%%%%%%%%'
    print '#################################'
    mean_acc = 0
    mean_prec = 0
    mean_rec = 0
    mean_f1 = 0
    conf = numpy.zeros([5,5])
    for i in xrange(cv_rounds):
        mean_acc += results[name]['acc'][i]
        #print results[key]['report'][i].split('     ')
        mean_prec += float(results[name]['rep'][i].split('     ')[-4][2:])
        mean_rec += float(results[name]['rep'][i].split('     ')[-3][2:])
        mean_f1 += float(results[name]['rep'][i].split('     ')[-2][2:])
        conf += results[name]['conf'][i]
    mean_acc = mean_acc/float(cv_rounds)
    mean_prec = mean_prec/float(cv_rounds)
    mean_rec = mean_rec/float(cv_rounds)
    mean_f1 = mean_f1/float(cv_rounds)
    conf = conf/float(cv_rounds)
    print('Accuracy : {}'.format(mean_acc))
    print('Precision : {}'.format(mean_prec))
    print('Recall : {}'.format(mean_rec))
    print('F1 : {}'.format(mean_f1))
    print('Confusion matrix :\n {}'.format(conf))
    print '#################################'
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

%%%%%%%%%%%%%%%%  3grams  % %%%%%%%%%%%%%%%%%%%%%%%
#################################
Accuracy : 0.441563228438
Precision : 0.358
Recall : 0.443
F1 : 0.366
Confusion matrix :
 [[  7.2  13.3   0.4   0.    0. ]
 [  4.9  21.2   0.6   0.4   0. ]
 [  2.5   9.2   0.2   0.1   0. ]
 [  1.    2.8   0.1   0.1   0. ]
 [  0.2   0.8   0.    0.    0. ]]
#################################
%%%%%%%%%%%%%%%%  soac  % %%%%%%%%%%%%%%%%%%%%%%%
#################################
Accuracy : 0.36321532634
Precision : 0.354
Recall : 0.364
F1 : 0.341
Confusion matrix :
 [[  8.5   9.4   1.9   1.1   0. ]
 [  8.9  13.3   2.6   2.3   0. ]
 [  3.9   6.1   1.3   0.7   0. ]
 [  1.2   2.    0.3   0.5   0. ]
 [  0.3   0.4   0.2   0.1   0. ]]
#################################
%%%%%%%%%%%%%%%%  lsi  % %%%%%%%%%%%%%%%%%%%%%%%
#################################
Accuracy : 0.324606643357
Precision : 0.122
Recall : 0.327
F1 : 0.176
Confusion matrix :
 [[  4.1  12.6   0.    4.2   0. ]
 [  5.5  16.2   0.    5.4   0. ]
 [  2.4   7.

In [ ]:
gg.feature_count_

In [ ]:
for i in xrange(len(y_meta)):
    print "True Label: %s" % y_meta[i]
    prediction_string = ''
    for name, model in zip(base_model_names, pred_cv):
        prediction_string += ' %s:%s |' % (name, g.lab.inverse_transform(g.transformation(model).argmax(axis=1))[0])
    prediction_string += ' %s:%s' % ('Bernoulli', predict[i])
    print prediction_string[1:]

In [159]:
g = combinations.Combinator(scheme='majority')
g.fit(base_predictions_meta, y_meta)
pred_meta = g.lab.transform(base_predictions_meta)
pred_meta = g.ohe.transform(pred_meta.reshape(-1, 1)).todense().reshape(len(base_predictions_meta[0]), -1)
pred_cv = g.lab.transform(base_predictions_cv)
pred_cv = g.ohe.transform(pred_cv.reshape(-1, 1)).todense().reshape(len(base_predictions_cv[0]), -1)
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
gg =MultinomialNB()
gg.fit(pred_meta, y_meta)
gg.predict(pred_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Accuracy : 0.292307692308
Confusion matrix :
 [[ 0  0  3  1  0]
 [ 0  0 20  1  0]
 [ 0  7 18  1  1]
 [ 0  4  7  1  0]
 [ 0  0  0  1  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.00      0.00      0.00        21
      35-49       0.38      0.67      0.48        27
      50-64       0.20      0.08      0.12        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.19      0.29      0.22        65



In [65]:
s = []
l = [[base0,base1,base2]for base0,base1,base2 in zip(base_predictions_meta[0],base_predictions_meta[1],base_predictions_meta[2])]

In [167]:
len(base_predictions_meta[0]), len(y_meta)

(65, 65)

In [183]:
from sklearn.base import BaseEstimator, TransformerMixin
from pan.features import tokenization2

class Freq(BaseEstimator, TransformerMixin):
    
    """ Find the frequency of votes corresponding to specific classes.
        It does not learn anything new. Just counting on the evaluation
        dataset.
        
        Args:
    """
    
    
    def __init__(self):
        
        from sklearn.preprocessing import LabelEncoder, OneHotEncoder
        from sklearn.naive_bayes import BernoulliNB
        
        self.rule_book = None
        self.rule_index = {}
        self.transformed = None
        self.labels = None
        self.num_models = None
        self.num_labels = None
        self.lab = LabelEncoder()
        self.ohe = OneHotEncoder()
        self.Bernoulli = BernoulliNB()
        
        
        
    def fit(self, X, y):
        """Expects a list of string inputs as X and a list of lists containing
           the prediciton of each model for y"""
        
        import numpy
        
        X_tr, y_tr = self.fit_encoders(X, y)
        self.transformed = X_tr
        tmp = numpy.zeros([1, self.num_labels])
        j = 0
        self.rule_book = numpy.copy(tmp)
        for i, x in enumerate(X_tr):
            x = str(x)
            #print x
            #print self.rule_book
            if x in self.rule_index.keys():
                #print y_tr[i].argmax(axis=1)
                #print self.rule_book[self.rule_index[x], y_tr.argmax(axis=1)].shape
                self.rule_book[self.rule_index[x], y_tr[i].argmax(axis=1)] += 1
                #print self.rule_book[self.rule_index[x], :]
            else:
                self.rule_index[x] = j
                self.rule_book = numpy.vstack((self.rule_book, tmp))
                #self.rule_book = numpy.delete(self.rule_book, 0, 0)
                self.rule_book[self.rule_index[x], y_tr[i].argmax(axis=1)] += 1
                j += 1
        self.num_rules = len(list(self.rule_index.keys()))
        self.rule_book = numpy.delete(self.rule_book, self.rule_book.shape[0]-1, 0)
#         print "Fitted"
#         import pprint
#         pprint.pprint(self.rule_index)
#         print self.rule_book
        # print X_tr
        self.Bernoulli.fit(X_tr, y)
        return
            
        
        
    def fit_encoders(self, X, y):

        import numpy

        self.num_labels = len(set(y))
        self.labels = sorted(list(set(y)))
        N_samples = len(y)
        if type(X) is numpy.array:
            y = y.reshape(-1, 1)
        else:
            y = numpy.array(y).reshape(-1, 1)
        y = self.lab.fit_transform(y).reshape(-1, 1)
        y = self.ohe.fit_transform(y).todense()
        X = self.lab.transform(X)
        X = self.ohe.transform(X.T.reshape(-1, 1)).todense().reshape(N_samples, -1)
        self.num_models = int(X.shape[1] / self.num_labels)
        return X, y
    
    def transformation(self, X, y):
        
        import random
        #from sklearn.
        predictions = []
        print len(X), len(y)
        for i, x in enumerate(X):
            x1 = str(x)
            if x1 in self.rule_index.keys():
#                 print "True"
#                 print y[i]
                #print self.rule_book[self.rule_index[x],:]
                #print self.rule_book[self.rule_index[x],:].argmax(axis=0)
                predictions.append(self.labels[self.rule_book[self.rule_index[x1],:].argmax(axis=0)])
#                 print "Predicted"
#                 print self.rule_book[self.rule_index[x1],:]
#                 print predictions[-1]
            else:
                predictions.append(random.choice(self.labels))
                scores = []
                for x_ in self.transformed:
                    pass
                #print [x]
                #print type(x)
                    # print [x[0]]
        predictions = []
        #for x in X:
        #    predictions.append(self.Bernoulli.predict([x]))
        predictions = self.Bernoulli.predict(X)    
        #predictions.append(self.Bernoulli.predict([]))
        return predictions
    
    
    def transform(self, X, y):

        import numpy

        if type(X[0]) is numpy.array:
            N_samples = X[0].shape[0]
        else:
            N_samples = len(X[0])
        X = self.lab.transform(X)
        X = self.ohe.transform(X.reshape(-1, 1)).todense().reshape(N_samples, -1)
        #print len(X), len(y)
        prediction = self.transformation(X, y)
        #prediction = self.lab.inverse_transform(prediction.argmax(axis=1))
        return prediction

    def predict(self, X, y):
        return self.transform(X, y)

    def score(self, X, y_true):

        from sklearn.metrics import accuracy_score

        y_pred = self.predict(X)
        return accuracy_score(y_true, y_pred, normalize=True)
            
            
        
        

In [184]:
a = Freq()
a.fit(base_predictions_meta, y_meta)
predict = a.predict(base_predictions_cv, y_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

66 66
Accuracy : 0.257575757576
Confusion matrix :
 [[ 0  2  0  2  0]
 [ 5  9  2  3  2]
 [11  9  4  4  0]
 [ 3  4  1  4  0]
 [ 1  0  0  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.38      0.43      0.40        21
      35-49       0.57      0.14      0.23        28
      50-64       0.31      0.33      0.32        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.42      0.26      0.28        66



In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

class SubSpaceEnsemble3(BaseEstimator, TransformerMixin):
    
    """ Utilizing the neighborhood in all representations and also ground truth model.
        Implementing a weighted voting scheme."""

    def __init__(self, models, k=3, weights= [2,1,3,0.7]):
        from sklearn.feature_extraction.text import CountVectorizer
        
        if (not models):
            raise AttributeError('Models expexts a dictonary of models \
              containg the predictions of y_true for each classifier.\ ')
        else:
            self.models = models
            # self.cv_scores = cv_scores
            self.k = k
            self.weights = weights
            self.ind2names = {}
            for i, name in enumerate(models.keys()):
                self.ind2names[i] = name
            self.counter = CountVectorizer()
            self.representations = []
            self.meta = None
            self.predictions = []
            self.true = []
            self.doc_terms = None
            self.tree = None
            self.experts = []
        

    def fit(self, X_cv, y_true=None, weights=None):
        
        from sklearn.neighbors import BallTree
        import random

        if y_true is None:
            raise ValueError('we need y labels to supervise-fit!')
        else:
            parameters = {
                    'input': 'content',
                    'encoding': 'utf-8',
                    'decode_error': 'ignore',
                    'analyzer': 'word',
                    'stop_words': 'english',
                    # 'vocabulary':list(voc),
                    #'tokenizer': tokenization,
                    #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                    #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                    'max_df': 1.0,
                    'min_df': 1,
                    'max_features':None
                }
            t0 = time.time()
            self.counter.set_params(**parameters)
            self.doc_terms = self.counter.fit_transform(X_cv).toarray()
            self.tree = BallTree(self.doc_terms, leaf_size=20)
            predictions = []
            for name, model in self.models.iteritems():
                predictions.append(model.predict(X_cv))
                #print len(predictions[-1])
                transf = model.steps[0][1].transform(X_cv)
                if hasattr(transf, "toarray"):
                    #print 'Exei'
                    self.representations.append(transf.toarray())
                else:
                    self.representations.append(transf)
            self.predictions = predictions
            self.true = y_true
            count = 0
            #print self.expert_scores
            #print self.experts
            print('Fit took: %0.3f seconds') % (time.time()-t0)
            return self

    def predict(self, X):
        # print "PRedict"
        # print X.shape
        X_transformed = self.counter.transform(X).toarray()
        #print type((X_transformed)[0])
        #print X_transformed.shape
        #return 0
        y_pred = []
        t0 = time.time()
        for i in range(0, X_transformed.shape[0]):
            #print X_transformed[i,:].shape
            dist, neigbors_indexes = self.tree.query(X_transformed[i,:].reshape(1,-1), self.k)  
            #print 'Sample ' + y_real[i]
            #print dist
            #print type(dist)
            #print neigbors_indexes[0]
            #print dist
            #best_model_ind = self.expert_decision(neigbors_indexes[0])
            #pass
            y_pred.append(self.expert_decision(neigbors_indexes[0],  dist, X[i]))
            
            #y_pred.append(self.models[self.ind2names[best_model_ind]].predict([X[i]])[0])
        #print y_pred
        print('Predict took: %0.3f seconds') % (time.time()-t0)
        return y_pred

    def score(self, X, y, sample_weight=None):

        from sklearn.metrics import accuracy_score
        return accuracy_score(y, self.predict(X), normalize=True)
        #return self.svc.score(self.transform_to_y(X), y, sample_weight)


    def expert_decision(self, neigbors_indexes, dist, x_sample):

        from sklearn.metrics import accuracy_score
        from collections import Counter
        from sklearn.neighbors import BallTree
        
        models_pred = []
        models_neig_pred = []
        acc = []
        t0 = time.time()
        neigbors_true = [self.true[n_i] for n_i in neigbors_indexes]
        #print('Neighbors per sample: %0.4f seconds') % (time.time()-t0)
#         print 'True'
#         print neigbors_true
        sample_predictions = []
        total_pred = []
        weights = {}
        weights['true'] = self.weights[2]
        weights['models_n'] = []
        weights['models'] = []
        for model_i in xrange(len(self.models.values())):
            ModelTree = BallTree(self.representations[model_i])
            temp_trans = self.models[self.ind2names[model_i]].steps[0][1].transform([x_sample])
            if hasattr(temp_trans, 'toarray'):
                temp_trans = temp_trans.toarray()
            _, model_neig = ModelTree.query(temp_trans, self.k)
            model_neig_pred = []
            for model_n_i in model_neig[0].tolist():
                model_neig_pred.append(self.predictions[model_i][model_n_i])
            models_neig_pred.append(model_neig_pred)
            model_pred = []
            for n_i in neigbors_indexes:
                model_pred.append(self.predictions[model_i][n_i])
            models_pred.append(model_pred)
            acc.append(accuracy_score(neigbors_true, model_pred, normalize=True))
            if acc[-1] >self.weights[3]:
                # Adding neighbors predictions
                weights['models_n'].append(int(self.weights[1]/float((1-acc[-1])+0.01)))
                total_pred.extend([pred for j in xrange(weights['models_n'][-1]) for pred in model_pred])
                #print('Predicting Neighbors per sample: %0.4f seconds') % (time.time()-t0)
                # Adding sample prediction
                sample_predictions.append(self.models[self.ind2names[model_i]].predict(x_sample)[0])
                weights['models'].append(int(self.weights[0]/float((1-acc[-1])+0.01))) 
                total_pred.extend([sample_predictions[-1] for j in xrange(weights['models'][-1])])
                total_pred.extend([pred for j in xrange(weights['models'][-1]) for pred in model_neig_pred])
            #print len(x_sample)
            #print self.ind2names[model_i]
            
#                 print 'Model: ' + self.ind2names[model_i] + ' Accuracy: ' + str(accuracy_score(neigbors_true, model_pred, normalize=True))
#                 print 'Predictions'
#                 print model_pred
#                 print 'Representations'
#                 print model_neig_pred
#                 print 'Sample prediction: ' + str(sample_predictions[-1])
        total_pred.extend([n for i, n in enumerate(neigbors_true) for j in xrange(int(weights['true']*(self.k-i)))])
        #print('creating votes: %0.4f seconds') % (time.time()-t0)
        data = Counter(total_pred)
        #data = Counter([k for pred in models_pred for k in pred])
#         print data
#         best_model_ind = acc.index(max(acc))
#         print 'Total pred: ' + str(data.most_common(1)[0][0])
        #print len(total_pred)
        #return best_model_ind
        return data.most_common(1)[0][0]

In [63]:
import pandas
p1 = models_for_space['3grams'].predict(X_meta)
df1 = pandas.DataFrame(numpy.hstack((p1.reshape(-1,1), numpy.array(y_cv).reshape(-1,1))), columns=['3grams', 'True'])

In [109]:
g = df1[df1['3grams']==df1['True']]
#print g
g[g['True'] == '35-49'].shape[0]
g.index.rename(['lala' + str(i) for i in xrange(g.shape[0])])

Int64Index([ 1,  2,  5,  9, 14, 15, 21, 22, 23, 26, 29, 31, 33, 35, 39, 40, 45,
            47, 49, 51, 53, 58, 60, 61, 63, 64],
           dtype='int64', name=[u'lala0', u'lala1', u'lala2', u'lala3', u'lala4', u'lala5', u'lala6', u'lala7', u'lala8', u'lala9', u'lala10', u'lala11', u'lala12', u'lala13', u'lala14', u'lala15', u'lala16', u'lala17', u'lala18', u'lala19', u'lala20', u'lala21', u'lala22', u'lala23', u'lala24', u'lala25'])

In [18]:
space2 = SubSpaceEnsemble3(models_for_space, k=3, weights= [6,3,2,0.7])
#2,1,3,0.7
space2.fit(X, y)
predict = space2.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Fit took: 18.552 seconds
Predict took: 78.486 seconds
Accuracy : 0.6
Confusion matrix :
 [[ 1  1  0  2  0]
 [ 2 11  4  3  0]
 [ 0  4 22  2  0]
 [ 0  0  7  5  0]
 [ 0  1  0  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.33      0.25      0.29         4
      25-34       0.65      0.55      0.59        20
      35-49       0.67      0.79      0.72        28
      50-64       0.42      0.42      0.42        12
      65-xx       0.00      0.00      0.00         1

avg / total       0.58      0.60      0.59        65



In [17]:
for name in all_models_names:
    print name
    print results[name]['acc'][1]

3grams
0.363636363636
soac
0.333333333333
lsi
0.318181818182
votingf
0.393939393939
votingh
0.333333333333
space3
0.878787878788
meta
0.242424242424
majority
0.424242424242
weights
0.424242424242
accuracy
0.424242424242
optimal
0.409090909091


In [23]:
from sklearn.base import BaseEstimator, TransformerMixin

class SubSpaceEnsemble3_w(BaseEstimator, TransformerMixin):
    
    """ Utilizing the neighborhood in all representations and also ground truth model.
        Implementing a weighted voting scheme. Finding Optimal weights also!"""

    def __init__(self, models, cv_scores, k=3, scheme = 'weights', weights=[6,3,2,0.7]):
        from sklearn.feature_extraction.text import CountVectorizer
        
        if (not models) or (not cv_scores):
            raise AttributeError('Models expexts a dictonary of models \
              containg the predictions of y_true for each classifier.\
              cv_score expects a list len(models.keys()) with the\
              cross validation scores of each model')
        else:
            self.models = models
            self.cv_scores = cv_scores
            self.k = k
            self.scheme = scheme
            self.weights = None
            self.ind2names = {}
            for i, name in enumerate(models.keys()):
                self.ind2names[i] = name
            self.counter = CountVectorizer()
            self.representations = []
            self.meta = None
            self.predictions = []
            self.true = []
            self.doc_terms = None
            self.tree = None
            self.experts = []
            if self.scheme == 'weights':
                if type(self.weights) in (numpy.array, numpy.ndarray):
                    pass  # It is from the optimization part
                else:
                    if not(self.weights):
                        print "Need weights for this scheme!"
                self.weights = weights
                weights_string = " %.2f |" * len(self.weights) % tuple(self.weights)
                # print "Using given weights: | %s" % weights_string
            elif self.scheme == 'optimal':
                # print "Will find the weights after fitting"
                pass
            else:
                self.weights = [6,3,2,0.7]
        

    def fit(self, X, X_cv, y = None, y_true=None, weights=None):
        
        from sklearn.neighbors import BallTree
        import random

        if y_true is None:
            raise ValueError('we need y labels to supervise-fit!')
        else:
            parameters = {
                    'input': 'content',
                    'encoding': 'utf-8',
                    'decode_error': 'ignore',
                    'analyzer': 'word',
                    'stop_words': 'english',
                    # 'vocabulary':list(voc),
                    #'tokenizer': tokenization,
                    #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                    #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                    'max_df': 1.0,
                    'min_df': 1,
                    'max_features':None
                }
            t0 = time.time()
            self.counter.set_params(**parameters)
            self.doc_terms = self.counter.fit_transform(X + X_cv).toarray()
            self.tree = BallTree(self.doc_terms, leaf_size=20)
            predictions = []
            for name, model in self.models.iteritems():
                predictions.append(model.predict(X + X_cv))
                #print len(predictions[-1])
                transf = model.steps[0][1].transform(X + X_cv)
                if hasattr(transf, "toarray"):
                    #print 'Exei'
                    self.representations.append(transf.toarray())
                else:
                    self.representations.append(transf)
            self.predictions = predictions
            self.true = y + y_true
            count = 0
            #print self.expert_scores
            #print self.experts
            if self.scheme=='optimal':
                self.find_weights(X, X_cv, y, y_cv)
                weights_string = " %.2f |" * len(self.weights) % tuple(self.weights)
            print('Fit took: %0.3f seconds') % (time.time()-t0)
            return self
        
    def find_weights(self, X, X_cv, y_cv, y):

        import numpy

        from scipy.optimize import minimize

        w = [6,3,2,0.7]
        bnds = tuple([(0, None) for i in xrange(len(w))])
        a = minimize(self.f, w, args=(SubSpaceEnsemble3_w, X, X_cv, y, y_cv), method='L-BFGS-B', bounds=bnds)
        self.weights = list(a.x)
        return

    def f(self, w, SubSpaceEnsemble3_w, X, X_cv, y, y_cv):
        gg = SubSpaceEnsemble3_w(self.models, self.cv_scores, self.k, scheme= 'weights', weights= w)
        gg.fit(X, X_cv, y, y_cv)
        score = 1 - gg.score(x, y)
        # print 'Weights'
        # print w
        # print 'Score: ' + str(score)
        return score

    def predict(self, X):
        
        # print "PRedict"
        # print X.shape
        X_transformed = self.counter.transform(X).toarray()
        #print type((X_transformed)[0])
        #print X_transformed.shape
        #return 0
        y_pred = []
        t0 = time.time()
        for i in range(0, X_transformed.shape[0]):
            #print X_transformed[i,:].shape
            dist, neigbors_indexes = self.tree.query(X_transformed[i,:].reshape(1,-1), self.k)  
            # print 'Sample ' + y_real[i]
            #print neigbors_indexes[0]
            #print dist
            #best_model_ind = self.expert_decision(neigbors_indexes[0])
            y_pred.append(self.expert_decision(neigbors_indexes[0],  X[i]))
            
            #y_pred.append(self.models[self.ind2names[best_model_ind]].predict([X[i]])[0])
        #print y_pred
        print('Predict took: %0.3f seconds') % (time.time()-t0)
        return y_pred

    def score(self, X, y, sample_weight=None):

        from sklearn.metrics import accuracy_score
        return accuracy_score(y, self.predict(X), normalize=True)
        #return self.svc.score(self.transform_to_y(X), y, sample_weight)


    def expert_decision(self, neigbors_indexes, x_sample):

        from sklearn.metrics import accuracy_score
        from collections import Counter
        from sklearn.neighbors import BallTree
        
        models_pred = []
        models_neig_pred = []
        acc = []
        t0 = time.time()
        neigbors_true = [self.true[n_i] for n_i in neigbors_indexes]
        #print('Neighbors per sample: %0.4f seconds') % (time.time()-t0)
#         print 'True'
#         print neigbors_true
        sample_predictions = []
        total_pred = []
        weights = {}
        weights['true'] = self.weights[2]
        weights['models_n'] = []
        weights['models'] = []
        for model_i in xrange(len(self.models.values())):
            ModelTree = BallTree(self.representations[model_i])
            temp_trans = self.models[self.ind2names[model_i]].steps[0][1].transform([x_sample])
            if hasattr(temp_trans, 'toarray'):
                temp_trans = temp_trans.toarray()
            _, model_neig = ModelTree.query(temp_trans, self.k)
            model_neig_pred = []
            for model_n_i in model_neig[0].tolist():
                model_neig_pred.append(self.predictions[model_i][model_n_i])
            models_neig_pred.append(model_neig_pred)
            model_pred = []
            for n_i in neigbors_indexes:
                model_pred.append(self.predictions[model_i][n_i])
            models_pred.append(model_pred)
            acc.append(accuracy_score(neigbors_true, model_pred, normalize=True))
            if acc[-1] >self.weights[3]:
                # Adding neighbors predictions
                weights['models_n'].append(int(self.weights[1]/float((1-acc[-1])+0.01)))
                total_pred.extend([pred for j in xrange(weights['models_n'][-1]) for pred in model_pred])
                #print('Predicting Neighbors per sample: %0.4f seconds') % (time.time()-t0)
                # Adding sample prediction
                sample_predictions.append(self.models[self.ind2names[model_i]].predict(x_sample)[0])
                weights['models'].append(int(self.weights[0]/float((1-acc[-1])+0.01))) 
                total_pred.extend([sample_predictions[-1] for j in xrange(weights['models'][-1])])
                total_pred.extend([pred for j in xrange(weights['models'][-1]) for pred in model_neig_pred])
            #print len(x_sample)
            #print self.ind2names[model_i]
            
#                 print 'Model: ' + self.ind2names[model_i] + ' Accuracy: ' + str(accuracy_score(neigbors_true, model_pred, normalize=True))
#                 print 'Predictions'
#                 print model_pred
#                 print 'Representations'
#                 print model_neig_pred
#                 print 'Sample prediction: ' + str(sample_predictions[-1])
        total_pred.extend([n for j in xrange(int(weights['true'])) for n in neigbors_true])
        #print('creating votes: %0.4f seconds') % (time.time()-t0)
        data = Counter(total_pred)
        #data = Counter([k for pred in models_pred for k in pred])
#         print data
#         best_model_ind = acc.index(max(acc))
#         print 'Total pred: ' + str(data.most_common(1)[0][0])
        #print len(total_pred)
        #return best_model_ind
        return data.most_common(1)[0][0]

In [24]:
space_k = SubSpaceEnsemble3_w(models_for_space, cv_scores, k=3, scheme='optimal', weights=[2,1,3,0.5])
space_k.fit(X,X_meta, y,y_meta)
predict = space_k.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Need weights for this scheme!
Fit took: 121.750 seconds


MemoryError: 

In [146]:
#import EnsembleDiversityTests
reload(EnsembleDiversityTests)
from EnsembleDiversityTests import DiversityTests, BaseClassifiers
#gg = DiversityTests(predictions[:-1], print_names[:-1], predictions[-1])
gg = DiversityTests(base_predictions_cv, base_model_names, y_cv)
#gg.print_report()
gg1 = BaseClassifiers(base_predictions_cv, base_model_names, y_cv, True)
#gg1.get_comparison_report()

In [147]:
gg1.get_per_class_accuracy()

Per Class Accuracy of the models!
        18-24      25-34      35-49      50-64  65-xx
3grams      0  47.619048  74.074074   0.000000      0
soac        0  52.380952  40.740741  16.666667      0
lsi       100   0.000000   0.000000   0.000000      0
